In [23]:
import sys
import numpy as np
import pandas as pd

In [24]:
df=pd.read_csv('../Consolidated_AE_mastersheet.csv')
df

/home/srishti/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (13,18,19,31,32,34,35,46) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,State_Name,Assembly_No,Constituency_No,Year,month,DelimID,Poll_No,Position,Candidate,Sex,...,Contested,Last_Party,Last_Party_ID,Last_Constituency_Name,Same_Constituency,Same_Party,No_Terms,Turncoat,Incumbent,Recontest
0,Andhra_Pradesh,3,1,1962.0,2,1,0,1,KIRTI CHANDRA DEO,M,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,False,False,False
1,Andhra_Pradesh,3,1,1962.0,2,1,0,2,DAKKATA PITAMBARU,M,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,False,False,False
2,Andhra_Pradesh,3,2,1962.0,2,1,0,1,GOUTHU LATCHANNA,M,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,False,False,False
3,Andhra_Pradesh,3,2,1962.0,2,1,0,2,MAJJI TULASIDASS,M,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,False,False,False
4,Andhra_Pradesh,3,2,1962.0,2,1,0,3,GANNI PADMANABHA RAO,M,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464607,West_Bengal,17,294,2021.0,5,4,0,3,ASIF EKBAL,M,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,False,False,False
464608,West_Bengal,17,294,2021.0,5,4,0,4,ANSARUL SAIEKH,M,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,False,False,False
464609,West_Bengal,17,294,2021.0,5,4,0,5,TARAK LET,M,...,2.0,BSP,16651.0,RAMPURHAT,False,True,0.0,False,False,True
464610,West_Bengal,17,294,2021.0,5,4,0,6,None Of The Above,NaN,...,NaN,NaN,5897.0,NaN,NaN,NaN,NaN,NaN,False,False


In [25]:
df['State_Name']=df['State_Name'].mask(df['State_Name']=='Uttar_Pradesh','Uttar Pradesh')

In [26]:
states=['Goa','Manipur','Punjab','Uttar Pradesh','Uttarakhand']

df_assembly=df.groupby(['State_Name'])['Assembly_No'].max().reset_index()

df_assembly=df_assembly[df_assembly['State_Name'].isin(states)]

df_assembly

,State_Name,Assembly_No
6,Goa,7
18,Manipur,12
25,Punjab,15
31,Uttar Pradesh,17
32,Uttarakhand,4


In [27]:
#NOTA calculation


df_n=df[ (df['State_Name'].isin(states)) & (df['Poll_No']==0) & (df['Party']=='NOTA')]

df_indivi=df_n.groupby(['State_Name','Assembly_No','Year'])['Votes'].sum()

df_indivi=df_indivi.reset_index()

df_total=df_n.groupby(['State_Name','Assembly_No','Year'])['Valid_Votes'].sum()

df_total=df_total.reset_index()

merged_nota=df_total.merge(df_indivi,how='left',on=['State_Name','Assembly_No','Year'])

merged_nota['Percentage']=np.nan
merged_nota['Percentage']=merged_nota['Votes']*100/merged_nota['Valid_Votes']

print(merged_nota)

merged_nota.to_csv('nota.csv')

      State_Name  Assembly_No    Year  Valid_Votes     Votes  Percentage
0            Goa            7  2017.0     916216.0   10919.0    1.191750
1        Manipur           12  2017.0    1657776.0    9062.0    0.546636
2         Punjab           15  2017.0   15443460.0  108456.0    0.702278
3  Uttar Pradesh           17  2017.0   86727767.0  757643.0    0.873588
4    Uttarakhand            4  2017.0    4975167.0   50426.0    1.013554


In [28]:
#Total Voter Turnout 

df_v=df[(df['State_Name'].isin(states))]

#Valid Votes, #use of max function is to get an integer which is same otherwise, using unique gives list 
df_valid_votes=df_v.groupby(['State_Name','Assembly_No','Year','Constituency_No','Poll_No'])['Valid_Votes'].unique().reset_index().explode('Valid_Votes')
df_valid_votes=df_assembly.merge(df_valid_votes, how='left', on=['State_Name','Assembly_No'])

#To check for duplicates
print("Duplicates rows, if any",df_valid_votes[df_valid_votes.duplicated(subset=['State_Name','Assembly_No','Constituency_No'], keep=False)==True])

df_valid_votes=df_valid_votes.groupby(['State_Name','Assembly_No','Year'])['Valid_Votes'].sum().reset_index()

#Electors 

df_electors=df_v.groupby(['State_Name','Assembly_No','Year','Constituency_No','Poll_No'])['Electors'].unique().reset_index().explode('Electors')
df_electors=df_assembly.merge(df_electors, how='left', on=['State_Name','Assembly_No'] )

#To check for duplicates
print("Duplicates rows, if any",df_electors[df_electors.duplicated(subset=['State_Name','Assembly_No','Constituency_No'], keep=False)==True])

df_electors=df_electors.groupby(['State_Name','Assembly_No','Year'])['Electors'].sum().reset_index()

#Merging dataframes
merged_turnout=df_electors.merge(df_valid_votes, how='left', on=['State_Name','Assembly_No','Year'])
merged_turnout['Percentage']=np.nan
merged_turnout['Percentage']=merged_turnout['Valid_Votes']*100/merged_turnout['Electors']

print('Merged Turnout calculated\n', merged_turnout)

Duplicates rows, if any Empty DataFrame
Columns: [State_Name, Assembly_No, Year, Constituency_No, Poll_No, Valid_Votes]
Index: []
Duplicates rows, if any Empty DataFrame
Columns: [State_Name, Assembly_No, Year, Constituency_No, Poll_No, Electors]
Index: []
Merged Turnout calculated
       State_Name  Assembly_No    Year     Electors  Valid_Votes  Percentage
0            Goa            7  2017.0    1111691.0     916216.0   82.416427
1        Manipur           12  2017.0    1914543.0    1657776.0   86.588601
2         Punjab           15  2017.0   20029499.0   15443460.0   77.103576
3  Uttar Pradesh           17  2017.0  141664587.0   86727767.0   61.220499
4    Uttarakhand            4  2017.0    7606332.0    4975167.0   65.408228


In [29]:
#Voter Turnout 
df_turnout=pd.read_csv('../ae_voter_turnouts.csv')
gender_voter=df_assembly.merge(df_turnout, how='left', on=['State_Name','Assembly_No'])
gender_voter=gender_voter.merge(merged_turnout[['State_Name','Assembly_No','Year','Percentage']], how='left',on=['State_Name','Assembly_No'])

In [30]:
gender_voter.drop(['Year_x'], axis=1, inplace=True)

gender_voter.rename({'Year_y':'Year'}, axis=1, inplace=True)

gender_voter['total']=gender_voter['total'].mask(gender_voter['total'].isna(), gender_voter['Percentage'])

gender_voter.to_csv('voter.csv')

In [31]:
#Party Performance 

df_party_con=df[(df['State_Name'].isin(states))].groupby(['State_Name','Assembly_No'])['Party'].unique().reset_index()

df_party_con['Contested_Count']=np.nan
df_party_con['Contested_Count']=df_party_con['Party'].str.len()

df_party_rep=df[(df['State_Name'].isin(states)) & (df['Position']==1)].groupby(['State_Name','Assembly_No'])['Party'].unique().reset_index()

df_party_rep['Represented_Count']=np.nan
df_party_rep['Represented_Count']=df_party_rep['Party'].str.len()

df_party=df_assembly.merge(df_party_con[['State_Name','Assembly_No','Contested_Count']], how='left', on=['State_Name','Assembly_No'])

df_party=df_party.merge(df_party_rep[['State_Name','Assembly_No','Represented_Count']], how='left', on=['State_Name','Assembly_No'])

print(df_party)

df_party.to_csv('party.csv')

      State_Name  Assembly_No  Contested_Count  Represented_Count
0            Goa            7               19                  6
1        Manipur           12               19                  7
2         Punjab           15               53                  5
3  Uttar Pradesh           17              304                  9
4    Uttarakhand            4               36                  3


In [32]:
#Constituencies 

df_constituencies=df[(df['State_Name'].isin(states))].groupby(['State_Name','Assembly_No','Constituency_Type'])['Constituency_No'].unique().reset_index()


df_constituencies['count']=np.nan

df_constituencies['count']=df_constituencies['Constituency_No'].str.len()

temp=df_constituencies.groupby(['State_Name','Assembly_No'])['count'].sum().reset_index()


df_constituencies=df_constituencies.merge(temp, how='left', on=['State_Name','Assembly_No'])

df_constituencies.rename({'count_x':'count_i','count_y':'total'}, axis=1, inplace=True)

df_constituencies=df_assembly.merge(df_constituencies, on=['State_Name','Assembly_No'], how='left')


print(df_constituencies)

df_constituencies.to_csv('constituencies.csv')


       State_Name  Assembly_No Constituency_Type  \
0             Goa            7               GEN   
1             Goa            7                SC   
2         Manipur           12               GEN   
3         Manipur           12                SC   
4         Manipur           12                ST   
5          Punjab           15               GEN   
6          Punjab           15                SC   
7   Uttar Pradesh           17               GEN   
8   Uttar Pradesh           17                SC   
9     Uttarakhand            4               GEN   
10    Uttarakhand            4                SC   
11    Uttarakhand            4                ST   

                                      Constituency_No  count_i  total  
0   [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...       39     40  
1                                                 [2]        1     40  
2   [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...       40     60  
3                                  

In [33]:
#Women Winner

df_women_winner=df[(df['State_Name'].isin(states)) & (df['Position']==1) & (df['Sex']=='F')].groupby(['State_Name','Assembly_No'])['Constituency_No'].unique().reset_index()

df_women_winner=df_assembly.merge(df_women_winner, how='left', on=['State_Name','Assembly_No'])

df_women_winner['count']=np.nan

df_women_winner['count']=df_women_winner['Constituency_No'].str.len()

print(df_women_winner)

df_women_winner.to_csv('women_winner.csv')

      State_Name  Assembly_No  \
0            Goa            7   
1        Manipur           12   
2         Punjab           15   
3  Uttar Pradesh           17   
4    Uttarakhand            4   

                                     Constituency_No  count  
0                                           [12, 27]      2  
1                                           [19, 50]      2  
2                           [5, 70, 77, 93, 94, 105]      6  
3  [22, 23, 31, 32, 38, 57, 64, 68, 90, 94, 140, ...     42  
4                               [28, 36, 45, 51, 59]      5  


In [34]:
#Newcomers

df_new_comers=df[(df['State_Name'].isin(states)) & (df['Position']==1) & (df['Contested']==1)].groupby(['State_Name','Assembly_No'])['Constituency_No'].unique().reset_index()

df_new_comers['count']=np.nan

df_new_comers['count']=df_new_comers['Constituency_No'].str.len()

df_new_comers=df_assembly.merge(df_new_comers, how='left', on=['State_Name','Assembly_No'])

print(df_new_comers)

df_new_comers.to_csv('newcomer.csv')


      State_Name  Assembly_No  \
0            Goa            7   
1        Manipur           12   
2         Punjab           15   
3  Uttar Pradesh           17   
4    Uttarakhand            4   

                                     Constituency_No  count  
0                     [6, 7, 10, 13, 27, 28, 34, 38]      8  
1      [4, 6, 8, 10, 15, 19, 23, 33, 35, 40, 41, 43]     12  
2  [2, 3, 4, 8, 14, 16, 18, 22, 24, 25, 26, 28, 3...     58  
3  [1, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 16, 17, ...    238  
4  [5, 6, 7, 8, 9, 10, 18, 29, 33, 36, 37, 39, 41...     20  
